In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".13"

In [2]:
import pandas as pd
# from model import Brain
from brain import BrainModel
from submodels import factories
import matplotlib.pyplot as plt
import pandas as pd
from itertools import accumulate
import numpy as np
from collections import defaultdict
from random import shuffle

from lib.score import (
    fate_corr, score_both_size_norm, shrink_and_align_stats, score_stats_norm
)
from lib.preprocess import *
from lib.callback import (
    cell_number_callback, progenitor_number_callback, neuron_number_callback,
    TargetPopulation, TagNumberCallback,
)
from lib.sde.grn.grn4 import GRNMain4 as GRNMain
from lib.sde.mutate import mutate_grn4 as mutate_grn
from lib.ga.utils import weighted_selection_one, normalize_fitness_values
from jf.utils.export import Exporter
from jf.autocompute.jf import O, L
from itertools import product
import jf.models.stringmodel as sm
from lib.analyser import show_curve, show_curve_progenitor

In [3]:
HISTORY = defaultdict(dict)
HALL_OF_FAME = []

In [4]:
_count = -1
def provide_id():
    global _count
    _count += 1
    return _count

In [5]:
REF = O(
    stats=pd.read_csv("output/results/setup_basic/export/ref_basic2.csv"),  # ref is a mean
    # fmetric=setup_ref_fmetric("output/results/setup_basic/export/ref_fmetric_tristate.csv"),
)

In [6]:
def individual_generator(id_=-1):
    return Solution(GRNMain(5, 0, 1), id_=id_)

In [7]:
class Solution:
    def __init__(self, grn, id_=0, parent=-1):
        self.id = id_
        self.grn = grn
        self.parent = parent
        self.fit = -1
        self.stats = None
        
    def copy(self, id_=0):
        return Solution(self.grn.copy(), id_=id_, parent=self.id)
        
    def mutate(self):
        mutate_grn(self.grn)

In [8]:
def score_bb_size(bb, ref, *args, **kwargs):
    s_prog = score_stats_norm(bb.stats, ref.stats, col_stats=f"progenitor_pop_size",
                     col_ref="progenitor_pop_size", norm=2.0, *args, **kwargs)
    s_all = score_stats_norm(bb.stats, ref.stats, col_stats=f"whole_pop_size",
                     col_ref="whole_pop_size", norm=2.0, *args, **kwargs)
    s_neuron = score_stats_norm(bb.stats, ref.stats, col_stats=f"neuron_pop_size",
                     col_ref="size_type_PostMitotic", col_norm_data="progenitor_pop_size",
                       col_norm_ref="progenitor_pop_size", norm=2.0, *args, **kwargs)
    # print(s_prog, s_all, s_neuron)
    return s_prog + s_all + 0.1 * s_neuron

In [9]:
def setup_tag(cp):
    indexes = list(cp.base_population.keys())
    shuffle(indexes)
    splits = np.array_split(indexes, 3)
    for i, ls in enumerate(splits):
        for idx in ls:
            cp.base_population[idx].tag["subbrain"] = i

In [10]:
def get_bb(prun, grn):
    ccls = factories["grn4"](grn=grn)
    callbacks = dict(
        progenitor_pop_size=progenitor_number_callback,
        whole_pop_size=cell_number_callback,
        neuron_pop_size=neuron_number_callback,
    )
    bb = BrainModel(time_step=0.5, verbose=False, start_population=prun.size, max_pop_size=2e3,
            cell_cls=ccls, end_time=prun.end_time, start_time=50, silent=True, opti=True,
              run_tissue=False, monitor_callbacks=callbacks, tag_func=setup_tag)
    return bb

In [11]:
def run_grn(prun, grn):
    get_bb(prun, grn)
    bb.run()
    return bb

In [12]:
def fitness_multistep(prun, grn, steps):
    total_fitness = 0
    stop = False
    previous_time = None
    bb = get_bb(prun, grn)
    # first step
    for step in steps:
        if not bb.run_until(step.end_time):
            stop = True
        # score_step = score_both_size(bb.stats, prun.ref, max_step=step.end_time, min_step=previous_time)
        score_step = step.score_func(bb, prun.ref, max_step=step.end_time, min_step=step.start_time)
        fitness_step = 1.0 / score_step
        fitness_step = min(fitness_step, step.max_fitness)
        total_fitness += fitness_step
        if fitness_step < step.min_fitness or stop:
            return total_fitness, bb.stats
        else:
            previous_time = step.end_time
            step.passed()
        
    return total_fitness, bb.stats

def score_multistep(prun, stats, steps):
    total_fitness = 0
    stop = False
    previous_time = None
    # first step
    for i, step in enumerate(steps):
        # score_step = score_both_size(bb.stats, prun.ref, max_step=step.end_time, min_step=previous_time)
        score_step = step.score_func(bb, prun.ref, max_step=step.end_time, min_step=step.start_time)
        print(f"Score for step {i} is {score_step}")
        fitness_step = 1.0 / score_step
        fitness_step = min(fitness_step, step.max_fitness)
        total_fitness += fitness_step
        if fitness_step < step.min_fitness or stop:
            return total_fitness
        previous_time = step.end_time
        
    return total_fitness

In [13]:
def mean_sd_fitness(prun, grn, run=3):
    fitnesses = [fitness_multistep(prun, grn) for i in range(run)]
    return np.mean(fitnesses), np.std(fitnesses)

In [14]:
def multi_fitness(*args):
    fitnesses = [fitness_multistep(*args) for i in range(3)]
    scores = [x[0] for x in fitnesses]
    return fitnesses[scores.index(min(scores))]

In [15]:
def do_init(prun):
    return individual_generator(provide_id())

def do_fitness(prun, sol):
    fitness, stats = fitness_multistep(prun, sol.grn, prun.steps)
    return fitness, stats

def do_selection(prun, pop_fit, pop):
    if len(pop) < prun.min_pop:
        return individual_generator(provide_id())
    
    pop_fit = [p**prun.exponent for p in pop_fit]
    
    return weighted_selection_one(pop, pop_fit, individual_generator, new_fitness=0.5, id_=provide_id())[0]

def do_mutation(prun, sol):
    sol.mutate()
    return sol

In [16]:
class ObjectiveStep(O):
    start_time = 0
    end_time = 0
    max_fitness = 3
    min_fitness = 1
    name = ""
    _passed = False
    
    def reset(self):
        self._passed = False
    
    def passed(self):
        if self._passed:
            return
        print(f"Step {self.name} passed !")
        self._passed = True
    
example_steps = [
    ObjectiveStep(name="1", start_time=50, end_time=53, score_func=score_bb_size, min_fitness=0.3),
    ObjectiveStep(name="2", start_time=53, end_time=56, score_func=score_bb_size, min_fitness=0.3),
    ObjectiveStep(name="3", start_time=56, end_time=59, score_func=score_bb_size, min_fitness=0.3),
    ObjectiveStep(name="4", start_time=59, end_time=62, score_func=score_bb_size, min_fitness=0.),
    ObjectiveStep(name="5", start_time=62, end_time=65, score_func=score_bb_size, min_fitness=0.),
    ObjectiveStep(name="6", start_time=65, end_time=68, score_func=score_bb_size, min_fitness=0.),
    ObjectiveStep(name="7", start_time=68, end_time=71, score_func=score_bb_size, min_fitness=0.),
    ObjectiveStep(name="8", start_time=71, end_time=74, score_func=score_bb_size, min_fitness=0.),
    ObjectiveStep(name="9", start_time=74, end_time=77, score_func=score_bb_size, min_fitness=0.),
]

class ParamRun(O):
    pop_size = 10
    n_gen = 10
    current_gen = 0
    end_time = 83
    ref = REF
    min_pop = 20
    max_pop = 50

def get_prun(size=5, exponent=1):
    prun = ParamRun()
    prun.size = size
    prun.exponent = exponent
    prun.steps = example_steps
    return prun

In [17]:
def main(prun):
    prun.history = defaultdict(dict)
    # exporter = Exporter()
    best = 0
    sol = do_init(prun)
    pop = [sol]
    for generation in range(prun.n_gen * prun.pop_size):
        # args.generation = generation
        # objective.new_trial()
        fit, stats = do_fitness(prun, sol)
        sol.fit = fit
        sol.stats = stats
        
        # history
        # print(f"Fitness = {fit}", end="\t\t")
        if generation % 100 == 0:
            print(f"Step {generation}")
        if fit > best:
            print(f"++ Best {fit} for generation {generation}")
            best = fit
            
        monitor = sol
        prun.history[generation] = monitor
        # exporter(monitor, f"generation_g{generation}")
        
        # TODO get the stats associated with the best scores
        sub_pop = pop[-prun.max_pop:]
        sol = do_selection(prun, [s.fit for s in sub_pop], sub_pop)
            
        sol = do_mutation(prun, sol)
        pop.append(sol)
        
    print("OVER")
    return best

In [18]:
prun = get_prun()
# res = main(prun)

In [19]:
def get_score_parametrized(size=5, exponent=1, name=None):
    args = get_prun(size, exponent)
    args.name = name
    main(args)
    p1 = L(args.history.values()).fit
    idx = p1.index(max(p1))
    sol = args.history[idx]
    scores = [fitness_multistep(args, sol.grn, args.steps)[0] for i in range(10)]
    return np.mean(scores), np.std(scores), max(p1)

In [20]:
model = sm.StringModel("expstab_s{size}_exp{exponent}_i{i}")
keys = ("size", "exponent", "i")
gen = product([5, 7], [1, 2, 3], range(5))

In [ ]:
# main loop
exporter = Exporter(name="exp_exponent_140222")
for i, sample in enumerate(gen):
    name = model.fill(**dict(zip(keys, sample)))
    print(name, sample)
    if name in exporter.list():
        continue
    res = get_score_parametrized(*sample)
    exporter(res, name)

Exporting at output/exp_exponent_140222
expstab_s5_exp1_i0 (5, 1, 0)
Step 0
++ Best 0.065431901045508 for generation 0
Step 1 passed !
++ Best 0.5377065991219951 for generation 1
++ Best 0.6772321502686567 for generation 4
++ Best 0.6853471943521894 for generation 20
++ Best 1.426462735670617 for generation 23
++ Best 1.7396058024125576 for generation 25
Step 2 passed !
++ Best 1.924577250456512 for generation 96
OVER
expstab_s5_exp1_i1 (5, 1, 1)
Step 0
++ Best 0.0742513661275367 for generation 0
++ Best 0.09213673929615102 for generation 5
++ Best 0.09893765256651443 for generation 7
++ Best 0.16717184109495645 for generation 10
++ Best 0.182849824516638 for generation 52
++ Best 0.41976973056191125 for generation 96
OVER
expstab_s5_exp1_i2 (5, 1, 2)
Step 0
++ Best 0.025033224141972187 for generation 0
Step 3 passed !
Step 4 passed !
Step 5 passed !
Step 6 passed !
Step 7 passed !
Step 8 passed !
Step 9 passed !
++ Best 3.2344376659472847 for generation 1
++ Best 3.7908266003349933 fo

In [ ]:
show_curve(sol.stats, REF.stats)